CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [ ]:
!pip install alibi-detect


In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift


> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [3]:
from pytorch_tabular import TabularModel

num_features = [
    "dist_to_nearest_stn",
    "dist_to_dhoby",
    "degree_centrality",
    "eigenvector_centrality",
    "remaining_lease_years",
    "floor_area_sqm",
]

cat_features = [
    "month",
    "town",
    "flat_model_type",
    "storey_range",
]

features = num_features + cat_features

targets = ["resale_price"]

df = pd.read_csv("hdb_price_prediction.csv")
df_train = df[df["year"] <= 2019]
df_test_1 = df[df["year"] == 2022]
train = df_train[features + targets]
test_1 = df_test_1[features + targets]

model = TabularModel.load_model("model")

pred1 = model.predict(test_1)


def r2_score(pred):
    y_test = np.array(pred["resale_price"])
    y_pred = np.array(pred["resale_price_prediction"])

    ss_res = np.sum((y_test - y_pred) ** 2)
    ss_tot = np.sum((y_test - np.mean(y_test)) ** 2)

    return 1 - (ss_res / ss_tot)


print(f"Test R2: {r2_score(pred1)}")


2023-10-13 12:57:28,316 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
2023-10-13 12:57:28,325 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
c:\Users\tonyh\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Auto select gpus: [0]
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Output()

Test R2: 0.43884716817371616


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [4]:
df_test_2 = df[df["year"] == 2023]
test_2 = df_test_2[features + targets]

pred2 = model.predict(test_2)

print(f"Test R2: {r2_score(pred2)}")


Output()

Test R2: 0.16212582073719095


> Did model degradation occur for the deep learning model?


Yes, test R2 scores dropped rapidly from 2022 to 2023.



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [5]:
labels = ["No!", "Yes!"]

train_samples = train.sample(1000, random_state=SEED)[features]
test_2_samples = test_2.sample(1000, random_state=SEED)[features]

categories_per_feature = {
    features.index(cat_feature): None for cat_feature in cat_features
}

cd = TabularDrift(
    train_samples.values, p_val=0.05, categories_per_feature=categories_per_feature
)

fpreds = cd.predict(test_2_samples.values, drift_type="feature")

for f in range(cd.n_features):
    stat = "Chi2" if f in list(categories_per_feature.keys()) else "K-S"
    fname = features[f]
    is_drift = fpreds["data"]["is_drift"][f]
    stat_val, p_val = fpreds["data"]["distance"][f], fpreds["data"]["p_val"][f]
    print(
        f"{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}"
    )


dist_to_nearest_stn -- Drift? No! -- K-S 0.035 -- p-value 0.561
dist_to_dhoby -- Drift? No! -- K-S 0.059 -- p-value 0.059
degree_centrality -- Drift? No! -- K-S 0.038 -- p-value 0.455
eigenvector_centrality -- Drift? No! -- K-S 0.056 -- p-value 0.084
remaining_lease_years -- Drift? Yes! -- K-S 0.163 -- p-value 0.000
floor_area_sqm -- Drift? Yes! -- K-S 0.062 -- p-value 0.041
month -- Drift? Yes! -- Chi2 430.336 -- p-value 0.000
town -- Drift? No! -- Chi2 33.178 -- p-value 0.127
flat_model_type -- Drift? Yes! -- Chi2 62.122 -- p-value 0.001
storey_range -- Drift? Yes! -- Chi2 27.842 -- p-value 0.010


> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


Concept drift.

> From your analysis via TabularDrift, which features contribute to this shift?


`remaining_lease_years`, `floor_area_sqm`, `month`, `flat_model_type`, and `storey_range`

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


Retrain the model on newer data (<= 2022).

In [6]:
df_val = df[df["year"] <= 2022]
val = df_val[features + targets]
train_2 = pd.concat([train, test_2], ignore_index=True)

model.fit(
    train=train_2,
    validation=val,
    seed=SEED,
)

pred3 = model.predict(test_2)

print(f"Test R2: {r2_score(pred3)}")


Global seed set to 42


2023-10-13 12:57:29,555 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-13 12:57:29,562 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-13 12:57:29,679 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-10-13 12:57:29,702 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
c:\Users\tonyh\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-10-13 12

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\tonyh\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\core\module.py:481: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
c:\Users\tonyh\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\core\module.py:481: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at e:\Dev\NTU-SC4001\PartB\.lr_find_54328c65-1510-4ffa-861b-6a9c6df308e3.ckpt
Restored all states from the checkpoint file at e:\Dev\NTU-SC4001\PartB\.lr_find_54328c65-1510-4ffa-861b-6a9c6df308e3.ckpt
2023-10-13 12:57:37,237 - {pytorch_tabular.tabular_model:575} - INFO - Suggested LR

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.7 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

...

Output()

2023-10-13 13:00:20,696 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-13 13:00:20,697 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model


Output()

c:\Users\tonyh\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\utilities\cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


Test R2: 0.7261100840027195


### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |